In [2]:
#!pip install pyautogen


In [15]:
from dotenv import load_dotenv
import os
import autogen

In [31]:
load_dotenv(dotenv_path="../.env")
api_key = os.getenv("OPENAI_API_KEY")
# Define LLM config
llm_config = {
    "model": "gpt-4.1-nano",
    "api_key": api_key,  # Or use environment variable
    "temperature": 0,
}
# Create Agent 1: Question Asker
agent1 = autogen.AssistantAgent(
    name="QuestionAsker",
    llm_config=llm_config,
    system_message="""You are a curious agent. You always ask simple, logical questions to learn something new.
    After getting answers for one question, respond with TERMINATE to stop the conversation."""
)
# Create Agent 2: Answer Provider
agent2 = autogen.AssistantAgent(
    name="AnswerProvider",
    llm_config=llm_config,
    system_message="""You are a knowledgeable agent. 
    You answer questions concisely and correctly.
    """
)
# Start conversation
conversation = autogen.GroupChat(agents=[agent1, agent2], messages=[], max_round=2)
manager = autogen.GroupChatManager(groupchat=conversation, llm_config=llm_config)
# Initialize the conversation
agent1.initiate_chat(
    manager=manager,
    message="Can you tell me why the sky is blue?",
    recipient=agent2
    
)

QuestionAsker (to AnswerProvider):

Can you tell me why the sky is blue?

--------------------------------------------------------------------------------
AnswerProvider (to QuestionAsker):

The sky appears blue because molecules in the Earth's atmosphere scatter sunlight. Blue light has a shorter wavelength and is scattered more efficiently than other colors, making the sky look blue to our eyes.

--------------------------------------------------------------------------------
QuestionAsker (to AnswerProvider):

That's interesting! Do other planets in our solar system have atmospheres that cause their skies to look different colors?

--------------------------------------------------------------------------------
AnswerProvider (to QuestionAsker):

Yes, other planets have atmospheres that cause their skies to appear different colors. For example, Mars has a thin atmosphere with dust particles that give its sky a reddish or orange hue, while Venus has a thick, sulfuric acid cloud cover

ChatResult(chat_id=None, chat_history=[{'content': 'Can you tell me why the sky is blue?', 'role': 'assistant', 'name': 'QuestionAsker'}, {'content': "The sky appears blue because molecules in the Earth's atmosphere scatter sunlight. Blue light has a shorter wavelength and is scattered more efficiently than other colors, making the sky look blue to our eyes.", 'role': 'user', 'name': 'AnswerProvider'}, {'content': "That's interesting! Do other planets in our solar system have atmospheres that cause their skies to look different colors?", 'role': 'assistant', 'name': 'QuestionAsker'}, {'content': 'Yes, other planets have atmospheres that cause their skies to appear different colors. For example, Mars has a thin atmosphere with dust particles that give its sky a reddish or orange hue, while Venus has a thick, sulfuric acid cloud cover that makes its sky appear yellowish or orange.', 'role': 'user', 'name': 'AnswerProvider'}, {'content': 'TERMINATE', 'role': 'assistant', 'name': 'Question

In [32]:
#conversational agents using autogen

In [50]:
#!pip install agentops
#!pip install autogen

In [5]:
from pathlib import Path
import os
from autogen import ConversableAgent
from autogen.coding import  CodeBlock, LocalCommandLineCodeExecutor
import agentops
from dotenv import load_dotenv



load_dotenv(dotenv_path="../.env")
api_key = os.getenv("OPENAI_API_KEY")
work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

executor = LocalCommandLineCodeExecutor(work_dir=work_dir)

#Agent 1
# code_executor_agent = ConversableAgent(
#     name="code_executor_agent",
#     llm_config=False,
#     code_execution_config={
#         "executor": executor,
#     },
#     human_input_mode="NEVER",
# )

code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={
        "executor": "commandline-local",
    },
    human_input_mode="NEVER",
)

# The code writer agent's system message is to instruct the LLM on how to
# use the Jupyter code executor with IPython kernel.
code_writer_system_message = """
You have been given coding capability to solve tasks using Python code.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user 
to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, 
    download/read a file, 
    print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient 
    info is printed and the 
    task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the
    task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses 
code, and which step uses your 
language skill.When using code, you must indicate the script type in the code block. The user cannot provide any other
feedback or perform any other 
action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which
requires users to modify. 
Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block 
as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. 
Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
"""

import os

#Agent 2

code_writer_agent = ConversableAgent(
    "code_writer",
    system_message=code_writer_system_message,
    llm_config={"config_list": [{"model": "gpt-4", "api_key":api_key}]},
    code_execution_config=False,  # Turn off code execution for this agent.
    max_consecutive_auto_reply=2,
    human_input_mode="NEVER",
)

In [6]:
import pprint

chat_result = code_executor_agent.initiate_chat(
    code_writer_agent, message=" write python code for permutation for the word ALGEbRA. use optimised way to calculate it .I want final result count."
)

pprint.pprint(chat_result)
agentops.end_session("Success")

code_executor_agent (to code_writer):

 write python code for permutation for the word ALGEbRA. use optimised way to calculate it .I want final result count.

--------------------------------------------------------------------------------
code_writer (to code_executor_agent):

Sure, we can use the `itertools` library's `permutations` function to calculate the unique permutations. To optimize this, we'll take advantage of the fact that the word 'ALGEBRA' has some repeated letters. In this case, our permutation will be the permutation of the word length divided by the multiplication of the factorial of the count of each unique letter. Here's the Python code:

```python
from math import factorial
from collections import Counter

# The word to find permutations for
word = 'ALGEBRA'

# Count the occurrences of each unique letter
letter_counts = Counter(word)

# Calculate the number of unique permutations
permutations = factorial(len(word))
for count in letter_counts.values():
    permutati

In [8]:
# using ollama
!pip install ollama
!pip install fix-busted-json

In [3]:
from pathlib import Path
import os
from autogen import ConversableAgent
from autogen.coding import  CodeBlock, LocalCommandLineCodeExecutor
import agentops
from dotenv import load_dotenv



load_dotenv(dotenv_path="../.env")
api_key = os.getenv("OPENAI_API_KEY")
work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

executor = LocalCommandLineCodeExecutor(work_dir=work_dir)

#Agent 1
# code_executor_agent = ConversableAgent(
#     name="code_executor_agent",
#     llm_config=False,
#     code_execution_config={
#         "executor": executor,
#     },
#     human_input_mode="NEVER",
# )

code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={
        "executor": "commandline-local",
    },
    human_input_mode="NEVER",
)

# The code writer agent's system message is to instruct the LLM on how to
# use the Jupyter code executor with IPython kernel.
code_writer_system_message = """
You have been given coding capability to solve tasks using Python code.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user 
to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, 
    download/read a file, 
    print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient 
    info is printed and the 
    task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the
    task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses 
code, and which step uses your 
language skill.When using code, you must indicate the script type in the code block. The user cannot provide any other
feedback or perform any other 
action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which
requires users to modify. 
Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block 
as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. 
Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
"""

import os

#Agent 2

code_writer_agent = ConversableAgent(
    "code_writer",
    system_message=code_writer_system_message,
    llm_config={"config_list": [{"model": "llama3.1","api_type":"ollama","base_url": "http://localhost:11434"}]},
    code_execution_config=False,  # Turn off code execution for this agent.
    max_consecutive_auto_reply=2,
    human_input_mode="NEVER",
)

In [4]:
import pprint

chat_result = code_executor_agent.initiate_chat(
    code_writer_agent, message=" write python code for permutation for the word ALGEbRA. use optimised way to calculate it .I want final result count."
)

pprint.pprint(chat_result)
agentops.end_session("Success")

code_executor_agent (to code_writer):

 write python code for permutation for the word ALGEbRA. use optimised way to calculate it .I want final result count.

--------------------------------------------------------------------------------
code_writer (to code_executor_agent):

```python
from math import factorial

# filename: permutation.py

def permute(string):
    n = len(string)
    return factorial(n) / (factorial(n - 26 + 1))

word = "ALGEbRA"
print("Permutations of", word, "is:", int(permute(word)))
```

The `math.factorial` function calculates the factorial of a given number. Since we want all permutations of the word 'ALGEbRA', which has 7 letters, and there are two repeating characters 'E' (case-insensitive), but since our string is case-sensitive so it will be treated as one character, we calculate the permutation using `nPr` formula where n = total number of letters in the string.

This code block uses an optimized way to calculate permutations by utilizing pre-computed fac